# Connect Playlists and Locations

In [1]:
import pandas as pd
import time
import json
import googlemaps
import homebrew as hb
from sklearn.neighbors import NearestNeighbors

In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df = pd.read_json('artists_shows.json', lines=True)

In [4]:
print(df.shape)
df.head()

(134943, 3)


,artist,date,loc
0,2 Chainz,2019-06-30 05:30:00,"Las Vegas, NV, US"
1,2 Chainz,2019-07-12 11:00:00,"London, UK"
2,2 Chainz,2019-08-10 00:00:00,"Anaheim, CA, US"
3,2 Chainz,2019-08-24 22:00:00,"Atlanta, GA, US"
4,2 Chainz,2019-05-27 05:30:00,"Las Vegas, NV, US"


In [5]:
df.date.describe()

count                  134943
unique                  37136
top       2019-07-19 00:00:00
freq                      191
first     1973-09-29 00:00:00
last      2020-12-16 18:30:00
Name: date, dtype: object

In [6]:
df.columns = ['artist', 'date', 'location']    # rename loc -> location
domestics = df[df['location'].str.endswith('US')]    # drop any foreign gigs
recent_domestics = domestics[domestics['date'] >= '2015']    #  drop any gigs older than 5 yrs
locations = recent_domestics.copy()

In [7]:
unique_locations = locations['location'].unique()
f'There are {unique_locations.shape[0]} unique locations'

'There are 2023 unique locations'

Get lat longs from googlemaps

In [8]:
# unique_loc_w_latlong = []

# for i, location in enumerate(unique_locations):
#     row = {}
#     lat, lng = hb.get_latlong(location)
#     row['location'] = location
#     row['lat'] = lat
#     row['lng'] = lng
#     unique_loc_w_latlong.append(row)
#     if i % 50 == 0:
#         time.sleep(2)
#     if i % 100 == 0:
#         print(i)                # Slowish 

In [9]:
latlongs_df = pd.DataFrame(unique_loc_w_latlong)
latlongs_df.head()

NameError: name 'unique_loc_w_latlong' is not defined

In [61]:
shows_w_latlong = pd.merge(locations, latlongs_df, on='location', how='left')
print(locations.shape)
shows_w_latlong.shape

(75699, 3)


(75699, 5)

In [62]:
shows_w_latlong.to_json('shows_w_latlong.json')

### Look for Similar Artists

In [2]:
shows_w_latlong = pd.read_json('shows_w_latlong.json')

In [3]:
artists_playlists = pd.read_json('trimmed_artists_w_flags.json', lines=True)

In [4]:
artists_playlists['artist'] = artists_playlists['artist'].map(str.lower)
shows_w_latlong['artist'] = shows_w_latlong['artist'].map(str.lower)

In [61]:
similar_artist_model = hb.SimilarArtistModel(artists_playlists)

In [18]:
artists_playlists[artists_playlists['artist']=='van morrison']
# shows_w_latlong[shows_w_latlong['artist']=='van morrison']

,artist,date,location,lat,lng
71133,van morrison,2019-10-03 03:00:00,"Reno, NV, US",39.529633,-119.813803
71134,van morrison,2019-10-05 03:00:00,"Berkeley, CA, US",37.871593,-122.272747
71135,van morrison,2019-10-06 00:30:00,"Santa Barbara, CA, US",34.420831,-119.698190
71136,van morrison,2019-10-07 02:00:00,"Hollywood, CA, US",34.092809,-118.328661
71137,van morrison,2019-10-09 02:00:00,"Chula Vista, CA, US",32.640054,-117.084196
71138,van morrison,2019-04-25 16:00:00,"New Orleans, LA, US",29.951066,-90.071532
71139,van morrison,2019-04-26 00:00:00,"Chicago, IL, US",41.878114,-87.629798
71140,van morrison,2019-04-25 00:00:00,"Chicago, IL, US",41.878114,-87.629798
71141,van morrison,2019-04-24 00:00:00,"Chicago, IL, US",41.878114,-87.629798
71142,van morrison,2019-02-10 04:00:00,"Las Vegas, NV, US",36.169941,-115.139830


In [12]:
main_artist = input("Enter Artist:").lower()
num_artists = 2
if main_artist in shows_w_latlong['artist'].values:
    asimilar_artists = similar_artist_model.find_artists_sim_to(main_artist, n_neighbors=num_artists)

    for artist_tup in similar_artists:
        artist, cosine_sim = artist_tup
        print(f'Similar Band: {artist.upper()} - - Cosine Similarity: {cosine_sim}')
        new_spots = hb.pilfer_similar_artist(ref_df=shows_w_latlong,
                                             main_artist=main_artist,
                                             similar_artist=artist,
                                             max_new_cities=10)
        for spot in new_spots:
            print('\t', spot)
else:
    print(f'{main_artist.title()}: No Recent Shows OR Not In 20 Of Top Playlists')

Enter Artist: eagles


Similar Band: CARLY SIMON - - Cosine Similarity: 0.337


In [66]:
main_artist = input("Greg added this").lower()
num_artists = 5
similar_artists = similar_artist_model.find_artists_sim_to(main_artist, n_neighbors=num_artists)
if main_artist in shows_w_latlong['artist'].values:
    for artist, cosine_sim in similar_artists:
        print(f'Similar Band: {artist.upper()} - - Cosine Similarity: {cosine_sim}')
        new_spots = hb.pilfer_similar_artist(ref_df=shows_w_latlong,
                                             main_artist=main_artist,
                                             similar_artist=artist,
                                             max_new_cities=3)
        for spot in new_spots:
            print('\t', spot)
else:
    print(f'{main_artist.title()}: No Recent Shows OR Not In 20 Of Top Playlists')

Greg added this khalid


Similar Band: POST MALONE - - Cosine Similarity: 0.601
	 {'new': 'Nashville, TN, US', 'played': 'Memphis, TN, US', 'distance': 196.0}
	 {'new': 'Indianapolis, IN, US', 'played': 'Chicago, IL, US', 'distance': 165.0}
	 {'new': 'Pendleton, OR, US', 'played': 'Spokane, WA, US', 'distance': 152.0}
Similar Band: DOMINIC FIKE - - Cosine Similarity: 0.624
Similar Band: LAUV - - Cosine Similarity: 0.628
	 {'new': 'Fargo, ND, US', 'played': 'St. Paul, MN, US', 'distance': 222.0}
	 {'new': 'Nashville, TN, US', 'played': 'Memphis, TN, US', 'distance': 196.0}
	 {'new': 'Louisville, KY, US', 'played': 'Columbus, OH, US', 'distance': 189.0}
Similar Band: QUINN XCII - - Cosine Similarity: 0.632
	 {'new': 'Boise, ID, US', 'played': 'Spokane, WA, US', 'distance': 286.0}
	 {'new': 'Garden City, ID, US', 'played': 'Spokane, WA, US', 'distance': 285.0}
	 {'new': 'Santa Fe, NM, US', 'played': 'Morrison, CO, US', 'distance': 277.0}
Similar Band: BAZZI - - Cosine Similarity: 0.632
	 {'new': 'Indianapolis, IN

In [63]:
artist_in_question = 'the beatles'
similar_artist_model.find_similar_artist_venues(artist_in_question, shows_w_latlong)

'The Beatles: No Recent Shows OR Not In 20 Of Top Playlists'

In [65]:
similar_artist_model.find_artists_sim_to('ludwig van beethoven')

[('glenn gould', 0.388),
 ('royal stockholm philharmonic orchestra', 0.388),
 ('leif ove andsnes', 0.388),
 ('neeme järvi', 0.388),
 ('arvo pärt', 0.388),
 ('felix mendelssohn', 0.423),
 ('bamberg symphony', 0.423),
 ('royal philharmonic orchestra', 0.423),
 ('antonio vivaldi', 0.423),
 ('claudio abbado', 0.423),
 ('vladimir ashkenazy', 0.423),
 ('hector berlioz', 0.423),
 ('andré previn', 0.423),
 ('lorin maazel', 0.423),
 ('pyotr ilyich tchaikovsky', 0.484),
 ('johannes brahms', 0.5),
 ('erik satie', 0.567),
 ('claude debussy', 0.567),
 ('bergen philharmonic orchestra', 0.592),
 ('aaron copland', 0.592)]